# Week 04 Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm19prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.

Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

In [2]:
df = pd.read_csv('KNMI_20181231.txt.tsv', delimiter=",", header = 0, skiprows = 63)
df = df.drop(0)
# df = df[df['# STN'] == 270]
df = df.rename(columns = {'# STN':'station', 'YYYYMMDD': 'Date', '   TG':'Tmean', '   TN':'Tmin', '   TX':'Tmax'})
df = df.reset_index()
df = df.drop(columns =['   SQ','   DR', '   RH', 'index']) #  
df = df.sort_values(['Date'])
df.head(5)

c:\users\stelios\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0,2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,station,Date,Tmean,Tmin,Tmax
41903,248,20000101.0,,,
221688,319,20000101.0,71,58,88
303556,375,20000101.0,53,-8,79
214748,316,20000101.0,,,
268856,344,20000101.0,66,53,88


---

<a name='1'></a>
## Part 2: Clean the data

In [3]:
print(df.dtypes)

station     object
Date       float64
Tmean       object
Tmin        object
Tmax        object
dtype: object


The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [4]:
#replace cells with spaces to NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
#Check for mising values
value = df.isnull().values.any()
print('Is there any missing values?: {}!'.format(value))

Is there any missing values?: True!


In [6]:
#change data formats
df['station'] = df['station'].astype('int64')
df['Date'] = pd.to_datetime(df['Date'],  format='%Y%m%d')

#change data formats
df['Tmean'] = pd.to_numeric(df['Tmean'], errors = "coerce")
df['Tmax'] = pd.to_numeric(df['Tmax'], errors = "coerce")
df['Tmin'] = pd.to_numeric(df['Tmin'], errors = "coerce")
#remove leap year
# df = df[df['Date'].dt.is_leap_year == True]
# or?
df['month']= df['Date'].dt.month
df['day']= df['Date'].dt.day
df['year']= df['Date'].dt.year

df = df[~((df['month'] == 2) &
          (df['day'] == 29))] 


df =df.sort_values(['station', 'Date'])
df = df.set_index(['Date'])

#change temperatures to celcius degrees
df['Tmax'] = df['Tmax']/10
df['Tmin'] = df['Tmin']/10
df['Tmean'] = df['Tmean']/10


df.head()

,station,Tmean,Tmin,Tmax,month,day,year
Date,,,,,,,
2001-01-30,209,NaN,NaN,NaN,1,30,2001
2001-01-31,209,NaN,NaN,NaN,1,31,2001
2001-02-01,209,NaN,NaN,NaN,2,1,2001
2001-02-02,209,NaN,NaN,NaN,2,2,2001
2001-02-03,209,NaN,NaN,NaN,2,3,2001


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 331078 entries, 2001-01-30 to 2018-12-31
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   station  331078 non-null  int64  
 1   Tmean    237394 non-null  float64
 2   Tmin     237395 non-null  float64
 3   Tmax     237395 non-null  float64
 4   month    331078 non-null  int64  
 5   day      331078 non-null  int64  
 6   year     331078 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 20.2 MB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [8]:
df_multipleyears = df[df['year'] < 2018]
df_2018 = df[df['year'] == 2018]

df_2018
# df_multipleyears

,station,Tmean,Tmin,Tmax,month,day,year
Date,,,,,,,
2018-01-01,209,NaN,NaN,NaN,1,1,2018
2018-01-02,209,NaN,NaN,NaN,1,2,2018
2018-01-03,209,NaN,NaN,NaN,1,3,2018
2018-01-04,209,NaN,NaN,NaN,1,4,2018
2018-01-05,209,NaN,NaN,NaN,1,5,2018
...,...,...,...,...,...,...,...
2018-12-27,391,1.2,-1.8,4.7,12,27,2018
2018-12-28,391,0.7,-2.9,3.0,12,28,2018
2018-12-29,391,5.9,2.5,9.2,12,29,2018


In [9]:
all_stations = df.station.unique()
all_stations

array([209, 210, 215, 225, 235, 240, 242, 248, 249, 251, 257, 258, 260,
       265, 267, 269, 270, 273, 275, 277, 278, 279, 280, 283, 285, 286,
       290, 308, 310, 311, 312, 313, 315, 316, 319, 323, 324, 330, 331,
       340, 343, 344, 348, 350, 356, 370, 375, 377, 380, 391], dtype=int64)

## df_multipleyears.head()

In [10]:
def month_day(df_multipleyears):
    max_val = df_multipleyears.groupby(['month', 'day', 'station'])['Tmax'].max()
    min_val = df_multipleyears.groupby(['month', 'day', 'station'])['Tmin'].min()
    
    df_groupedbymonthday = pd.DataFrame()
    df_groupedbymonthday['min_val'] = min_val
    df_groupedbymonthday['max_val'] = max_val
    
    
#     print(df_groupedbymonthday)
    return(df_groupedbymonthday)

In [11]:
df_multi_years_stations = month_day(df_multipleyears)
df_multi_years_stations = df_multi_years_stations.reset_index()
df_multi_years_stations.head(5)

,month,day,station,min_val,max_val
0,1,1,209,NaN,NaN
1,1,1,210,-7.1,12.9
2,1,1,215,-0.5,8.1
3,1,1,225,NaN,NaN
4,1,1,235,-5.0,10.7


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [12]:
def smoothen(x, sigma):
    y = gaussian_filter1d(x, sigma)
    return y

In [13]:
def station(df, station):
    df = df[df['station'] == station]
    return df


---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [14]:
from scipy.interpolate import interpolate
from bokeh.models import Band, Range1d, Slider, CustomJS,Dropdown
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot, layout
from scipy.ndimage import gaussian_filter1d

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [15]:

def plot_measured(sigma, stations):
    
    def station(df, stations):
        df = df[df['station'] == stations]
        return df

    #
    xxx_station_2018 = station(df_2018, stations)
    xxx_station_multiyears = station(df_multi_years_stations, stations)
#     print(xxx_station_multiyears)
#     print(xxx_station_2018)
    x = xxx_station_2018.index
    y =  xxx_station_2018['Tmean']

    ynew = gaussian_filter1d(y, sigma)
    lower = gaussian_filter1d(np.array(xxx_station_multiyears['min_val']), sigma)
    upper = gaussian_filter1d(np.array(xxx_station_multiyears['max_val']), sigma)
    
    p = figure(
            x_axis_type='datetime',
            tools='',
            toolbar_location = None,
            plot_width=800,
            plot_height=600
          )


    source = ColumnDataSource({
            'base':x,
            'lower':lower,
            'upper':upper
            })



    band = Band(
                base = 'base',
                lower ='lower',
                upper ='upper',
                source = source,
                level = 'underlay',
                fill_alpha = 0.5,
                line_width = 1,
                line_color = 'purple'
               )


    p.y_range= Range1d(-20, 40)
#     p.line(x, y, color = 'red',alpha = 1, legend_label ="Temperature" )
    p.line(x, ynew, color = 'red',alpha = 0.7, legend_label ="Temperature" )

    p.title = 'Weather in Groningen'
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Temperature oC'

    p.add_layout(band)

    return(p)

plot_measured(3, 240)

Figure(id='1003', ...)

In [16]:
import panel as pn
pn.extension()
columns = all_stations.tolist()
sigma_range = np.arange(0.0000001, 100, 1)
menu, plot = pn.interact(plot_measured, sigma = sigma_range, stations = columns)




row = pn.Row( menu, plot)
row

Row
    [0] Column
        [0] DiscreteSlider(name='sigma', options=[1e-07, 1.0000001, ...], value=1e-07)
        [1] DiscreteSlider(formatter='%d', name='stations', options=[209, 210, 215, ...], value=209)
    [1] Row
        [0] Bokeh(Figure, name='interactive00101')